In [23]:
#LE LOGS E TRANSFORMA EM UM ARQUIVO COM LAYOUT BSM
import re
import pandas as pd
from tkinter import filedialog, Tk

def processar_arquivo_log(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
        linhas = arquivo.readlines()

    dados_extraidos = []

    padrao = re.compile(r"(?P<hora>\d{2}:\d{2}:\d{2}:\d{3})==>?#?(?P<evento>\w+)\s(?P<campos>.+)")

    for linha in linhas:
        match = padrao.search(linha)
        if '#' not in linha[0]:

            #hora = match.group('hora')
            evento = match.group('evento')
            #campos = match.group('campos').split('|')


            teste = linha.split("|")
            
            campo_dict = {campo.split('=')[0]: campo.split('=')[1] for campo in teste if '=' in campo}

            data = campo_dict['52'].split('-')[0]
            hora = (campo_dict['52'].split('-')[1])


            registro = {
                "Código Participante": '93',
                "Plataforma de negociação": "VALDI TRADER",
                "OMS": 'Trading Line Handler 11.1.2. 1049335',
                "Data": data,
                "Hora": hora,
                "ID da oferta": campo_dict.get('11', ''),
                "Id da oferta B3": campo_dict.get('17', ''),
                "Usuário": campo_dict.get('448', ''),
                "Código do Operador_Assessor": campo_dict.get('448', ''),
                "Código do cliente": campo_dict.get('1', ''),
                "Código Sessão": campo_dict.get('49', ''),
                "Status": evento,
                "Origem da oferta":campo_dict.get('50', ''),
                "Time Zone": ''
            }
            
            dados_extraidos.append(registro)

    return dados_extraidos

def salvar_como_csv(dados, caminho_saida):
    df = pd.DataFrame(dados)

    colunas_ordenadas = [
        "Código Participante", "Plataforma de negociação", "OMS", "Data", "Hora", 
        "ID da oferta", "Id da oferta B3", "Usuário", "Código do Operador_Assessor", 
        "Código do cliente", "Código Sessão", "Status", "Origem da oferta", "Time Zone"
    ]

    colunas_presentes = [col for col in colunas_ordenadas if col in df.columns]
    df[colunas_presentes].to_csv(caminho_saida, index=False, sep=';', encoding='utf-8')
    print(f"Arquivo salvo com sucesso em: {caminho_saida}")

def main():
    Tk().withdraw()
    caminhos_arquivos = filedialog.askopenfilenames(
        title="Selecione os arquivos de log", 
        filetypes=[("Arquivo de log", "*.log"), ("Todos os arquivos", "*.*")]
    )

    if not caminhos_arquivos:
        print("Nenhum arquivo selecionado.")
        return

    dados = []
    for caminho_arquivo in caminhos_arquivos:
        dados.extend(processar_arquivo_log(caminho_arquivo))

    if dados:
        caminho_saida = filedialog.asksaveasfilename(
            title="Salvar arquivo CSV",
            defaultextension=".csv",
            filetypes=[("Arquivo CSV", "*.csv")]
        )
        if caminho_saida:
            salvar_como_csv(dados, caminho_saida)
        else:
            print("Nenhum caminho de saída selecionado.")
    else:
        print("Nenhum dado foi extraído dos arquivos.")

if __name__ == "__main__":
    main()


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
#ITERADOR EXCEL CSV

import pandas as pd
from tkinter import Tk, filedialog

# Inicializa a janela do Tkinter (não será exibida)
Tk().withdraw()

# Seleção de múltiplos arquivos CSV
arquivos_selecionados = filedialog.askopenfilenames(
    title="Selecione os arquivos CSV",
    filetypes=[("Arquivos CSV", "*.csv")]
)

# Verifica se algum arquivo foi selecionado
if not arquivos_selecionados:
    print("Nenhum arquivo selecionado.")
else:
    # Lista para armazenar os DataFrames
    dataframes = []

    # Lê cada arquivo selecionado, usando ; como separador e latin1 como encoding
    for arquivo in arquivos_selecionados:
        try:
            df = pd.read_csv(arquivo, sep=';', encoding='latin1', dtype=str)
            dataframes.append(df)
            print(f"Arquivo {arquivo} carregado com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar o arquivo {arquivo}: {e}")

    # Combina todos os DataFrames sem perder a estrutura
    if dataframes:
        df_final = pd.concat(dataframes, ignore_index=True)

        # Exporta para um novo arquivo CSV com ponto e vírgula como separador
        df_final.to_csv('RESUORD_2024-10-01.csv', sep=';', index=False, encoding='latin1')

        print("Arquivo combinado criado com sucesso: 'arquivo_final.csv'")
    else:
        print("Nenhum arquivo válido foi carregado.")


In [ ]:
#ITERADOR DE TEXTO PARA EXCEL
import pandas as pd
from tkinter import filedialog, Tk
import os

# Função para selecionar múltiplos arquivos .txt
def selecionar_arquivos_txt():
    root = Tk()
    root.withdraw()  # Oculta a janela principal do Tkinter
    arquivos = filedialog.askopenfilenames(title="Selecione os arquivos de texto", filetypes=[("Arquivos Texto", "*.log")])
    return arquivos

# Função para processar os arquivos .txt e extrair as informações
def processar_arquivos_txt(arquivos):
    dados_combinados = []

    for arquivo in arquivos:
        with open(arquivo, 'r', encoding='utf-8') as file:
            linhas = file.readlines()  # Lê todas as linhas do arquivo
            for linha in linhas:
                # Se houver necessidade de remover espaços ou quebras de linha, podemos fazer isso:
                linha = linha.strip()
                
                # Adiciona cada linha à lista de dados
                dados_combinados.append([linha])  # Cada linha é uma lista, para adicionar como uma linha do Excel
    
    return dados_combinados

# Função para salvar os dados em um novo arquivo Excel
def salvar_novo_arquivo(dados):
    # Cria um DataFrame com os dados combinados
    df_novo = pd.DataFrame(dados)

    # Salva em um novo arquivo Excel
    caminho_arquivo = "dados_combinados.xlsx"
    df_novo.to_excel(caminho_arquivo, index=False, header=False)

    print(f"Arquivo Excel gerado em: {os.path.abspath(caminho_arquivo)}")

# Função principal
def main():
    arquivos = selecionar_arquivos_txt()
    if arquivos:
        dados = processar_arquivos_txt(arquivos)
        salvar_novo_arquivo(dados)
    else:
        print("Nenhum arquivo selecionado.")

# Executando o programa
if __name__ == "__main__":
    main()


In [ ]:
#DESCOMPACTADOR.

import os
import gzip
import tarfile
from tkinter import filedialog, Tk
import shutil

# Função para selecionar múltiplos arquivos compactados
def selecionar_arquivos():
    root = Tk()
    root.withdraw()  # Oculta a janela principal do Tkinter
    arquivos = filedialog.askopenfilenames(title="Selecione os arquivos compactados", filetypes=[("Arquivos Compactados", "*.gz;*.tar.gz")])
    return arquivos

# Função para descompactar arquivos .gz (não .tar.gz)
def descompactar_gz(arquivo):
    with gzip.open(arquivo, 'rb') as f_in:
        # O nome do arquivo descompactado será o mesmo sem a extensão '.gz'
        nome_arquivo = arquivo[:-3]
        with open(nome_arquivo, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Arquivo descompactado: {nome_arquivo}")

# Função para descompactar arquivos .tar.gz
def descompactar_tar_gz(arquivo):
    with tarfile.open(arquivo, "r:gz") as tar_ref:
        tar_ref.extractall()  # Extrai na pasta atual
    print(f"Arquivo descompactado: {arquivo}")

# Função principal que seleciona os arquivos e chama as funções de descompactação apropriadas
def descompactar_arquivos(arquivos):
    for arquivo in arquivos:
        if arquivo.endswith('.gz'):
            print(f"Descompactando {arquivo}...")
            descompactar_gz(arquivo)
        elif arquivo.endswith('.tar.gz'):
            print(f"Descompactando {arquivo}...")
            descompactar_tar_gz(arquivo)
        else:
            print(f"Formato de arquivo {arquivo} não suportado!")

# Função principal
def main():
    arquivos = selecionar_arquivos()
    if arquivos:
        descompactar_arquivos(arquivos)
    else:
        print("Nenhum arquivo selecionado.")

# Executando o programa
if __name__ == "__main__":
    main()



In [22]:
#LE ARQUIVO FIX E CONVERTE NO LAYOUT BSM

import re
import pandas as pd
from tkinter import filedialog, Tk

# Função para processar o arquivo FIX
def processar_arquivo_fix(caminho_arquivo):
    """Processa arquivos FIX e extrai os dados"""
    with open(caminho_arquivo, 'rb') as arquivo:  # Abrir como binário para lidar com SOH (0x01)
        linhas = arquivo.read().split(b'\x01')  # Dividir o arquivo pelo delimitador SOH (0x01)

    dados_extraidos = []

    for linha in linhas:
        if not linha.strip():
            continue  # Ignorar linhas vazias

        # Processa cada linha para extrair tag e valor
        campos = linha.decode('utf-8').split('|')  # Converter para string e dividir pelas tags

        campo_dict = {}
        for campo in campos:
            if '=' in campo:
                tag, valor = campo.split('=', 1)
                campo_dict[tag] = valor

        # Agora extrair as informações do campo_dict
        # Adapte as tags conforme o formato real do seu arquivo FIX
        data = campo_dict.get('52', '').split('-')[0]  # Extrai a data do campo '52'
        hora = campo_dict.get('52', '').split('-')[1] if '52' in campo_dict else ''  # Hora do campo '52'

        # Criar o registro a ser adicionado
        registro = {
            "Código Participante": '93',
            "Plataforma de negociação": "ATG",
            "OMS":'FALCON 1.0',
            "Data": data,
            "Hora": hora,
            "ID da oferta": campo_dict.get('11', ''),
            "Id da oferta B3": campo_dict.get('17', ''),
            "Usuário": campo_dict.get('448', ''),
            "Código do Operador_Assessor": campo_dict.get('448', ''),
            "Código do cliente": campo_dict.get('1', ''),
            "Código Sessão": campo_dict.get('49', ''),
            "Status": campo_dict.get('35', ''),  # Evento extraído da tag 35 (por exemplo)
            "Origem da oferta": campo_dict.get('50', ''),
            "Time Zone": ''
        }

        dados_extraidos.append(registro)

    return dados_extraidos

# Função para processar arquivo CSV (não modificada)
def processar_arquivo_csv(caminho_arquivo):
    """Processa arquivos CSV e retorna os dados"""
    df = pd.read_csv(caminho_arquivo, sep=';', encoding='utf-8')
    dados_extraidos = df.to_dict(orient='records')
    return dados_extraidos

# Função para salvar os dados em um arquivo CSV
def salvar_como_csv(dados, caminho_saida):
    """Salva os dados extraídos em um arquivo CSV"""
    df = pd.DataFrame(dados)

    # Organiza as colunas na ordem desejada
    colunas_ordenadas = [
        "Código Participante", "Plataforma de negociação", "OMS", "Data", "Hora", 
        "ID da oferta", "Id da oferta B3", "Usuário", "Código do Operador_Assessor", 
        "Código do cliente", "Código Sessão", "Status", "Origem da oferta", "Time Zone"
    ]

    colunas_presentes = [col for col in colunas_ordenadas if col in df.columns]
    df[colunas_presentes].to_csv(caminho_saida, index=False, sep=';', encoding='utf-8')
    print(f"Arquivo salvo com sucesso em: {caminho_saida}")

# Função principal
def main():
    Tk().withdraw()  # Oculta a janela principal do Tkinter
    
    caminhos_arquivos = filedialog.askopenfilenames(
        title="Selecione os arquivos", 
        filetypes=[("Arquivo FIX", "*.fix"), ("Arquivo CSV", "*.csv"), ("Todos os arquivos", "*.*")]
    )

    if not caminhos_arquivos:
        print("Nenhum arquivo selecionado.")
        return

    dados = []
    for caminho_arquivo in caminhos_arquivos:
        # Verifica a extensão do arquivo para processar adequadamente
        if caminho_arquivo.endswith('.fix'):
            dados.extend(processar_arquivo_fix(caminho_arquivo))  # Processa arquivos FIX
        elif caminho_arquivo.endswith('.csv'):
            dados.extend(processar_arquivo_csv(caminho_arquivo))  # Processa arquivos CSV

    if dados:
        caminho_saida = filedialog.asksaveasfilename(
            title="Salvar arquivo CSV",
            defaultextension=".csv",
            filetypes=[("Arquivo CSV", "*.csv")]  # Tipo de arquivo para salvar
        )
        if caminho_saida:
            salvar_como_csv(dados, caminho_saida)  # Salva os dados extraídos no formato CSV
        else:
            print("Nenhum caminho de saída selecionado.")
    else:
        print("Nenhum dado foi extraído dos arquivos.")

if __name__ == "__main__":
    main()


Nenhum dado foi extraído dos arquivos.


In [ ]:
#CRIA PASTAS 

import os
from datetime import datetime, timedelta

# Data inicial
start_date = datetime(2024, 10, 1)
end_date = datetime(2024, 12, 30)

# Criar pastas
current_date = start_date
while current_date <= end_date:
    date_string = current_date.strftime("%Y-%m-%d")
    os.makedirs(date_string, exist_ok=True)
    current_date += timedelta(days=1)